# Demonstration: using python for working with World Bank Data: plotting and data insertion

## Loading dependencies of the notebook

We use `pandas` to handle regular tabular data, `numpy` for numerical functionality, `wbgapi` to interact with World Bank Open Data's API and `geopandas` to handle geographical mapping.

In [ ]:
! bash -i -l -c "pip install -q wbgapi pycountry && pip install -q --upgrade nuvolos-odbc && conda install -q -y descartes geopandas ffmpeg"

In [ ]:
import wbgapi as wb
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np

## Exploring data on using `wbgapi`

`wbgapi` is a wrapper library for the World Bank Web API. For an introduction, please refer to the [vignette](https://blogs.worldbank.org/opendata/introducing-wbgapi-new-python-package-accessing-world-bank-data) of the package.

In [ ]:
FDI_GDP_PCT_info = wb.series.metadata.get('BX.KLT.DINV.WD.GD.ZS')

In [ ]:
FDI_GDP_PCT_info

In [ ]:
FDI_GDP_PCT_data = wb.data.DataFrame('BX.KLT.DINV.WD.GD.ZS', time=range(2000, 2020,), labels=True)
FDI_GDP_PCT_data.reset_index(inplace = True)

## Plotting the data

The plot we produce is quite similar to the one available directly on the website of the World Bank (see [source](https://data.worldbank.org/indicator/BX.KLT.DINV.CD.WD?view=map&year=2001)). 

In [ ]:
worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

It is possible to use `pandas` merge to merge a `geopandas` data frame with a regular `pandas` DataFrame. Once the appropriate cleanup and transformation is done, we call the `GeoDataFrame` constructor that uses the `geometry` column of `data_mapped` to as the geometry field.

In [ ]:
data_mapped = pd.merge(FDI_GDP_PCT_data, worldmap, left_on = 'economy', right_on = 'iso_a3')
data_mapped.drop(['pop_est', 'economy', 'gdp_md_est', 'Country'], axis = 1, inplace = True)
data_mapped = data_mapped.melt(id_vars = ['continent', 'iso_a3', 'name', 'geometry'])
data_mapped = gpd.GeoDataFrame(data_mapped)

In [ ]:
FDI_GDP_PCT_data

In [ ]:
worldmap.loc[worldmap['continent'] == 'Europe',:]

Upon closer inspection of the resulting mapped dataframe, it turns out that while certain countries do have shapes associated with them, their 3-letter ISO code is missing. This would result in e.g. France missing from our map!

## A small detour: obtaining correct ISO codes

In order to correct the world map information, we first rely on the `pycountry` package that comes with smart lookup functionality and country codes in both 2 and 3 letter ISO format. Given that we only have country names available for every country, we use the `search_fuzzy` method. The method prioritizes better matches, so we take the best match and its 3 letter ISO value.

In [ ]:
def map_country(x):
    import pycountry as pcc
    try:
        y = pcc.countries.search_fuzzy(x)[0].alpha_3
    except Exception as e:
        y = pd.NA
    return y

worldmap['iso_fuzzy'] = worldmap['name'].apply(lambda x: map_country(x))
worldmap['iso_a3_na'] = worldmap['iso_a3'].replace('-99', pd.NA)
worldmap['iso_merge'] = worldmap['iso_a3_na'].fillna(worldmap['iso_fuzzy'])

We re-execute the merging based on the enhanced worldmap.

In [ ]:
data_mapped = pd.merge(FDI_GDP_PCT_data, worldmap, left_on = 'economy', right_on = 'iso_merge')
data_mapped.drop(['pop_est', 'economy', 'gdp_md_est', 'Country', 'iso_fuzzy', 'iso_a3', 'iso_a3_na'], axis = 1, inplace = True)
data_mapped = data_mapped.melt(id_vars = ['continent', 'iso_merge', 'name', 'geometry'])
data_mapped = gpd.GeoDataFrame(data_mapped)

In [ ]:
data_mapped

In [ ]:
data_mapped_filter = data_mapped.loc[data_mapped.loc[:,'variable'] == 'YR2000',:]

### Addendum: loading other base layers

You can load other base layers using the `geopandas` package, see the documentation [here](https://geopandas.org/docs/user_guide/io.html). In particular, you can grab zipped ArcGIS repositories and load them directly into geopandas with the following convenience function.

In [ ]:
def get_and_load_map(url, local_zip_path):
    import urllib.request
    import os
    import geopandas as gpd
    try:
        urllib.request.urlretrieve(url, local_zip_path)
    except Exception as e:
        raise(e)   
    try:
        frame = gpd.read_file(f"zip:///{local_zip_path}")
    except Exception as e:
        raise(e)  
    return frame

In [ ]:
# An example of how to use it
yy = get_and_load_map('https://opendata.arcgis.com/datasets/2b93b06dc0dc4e809d3c8db5cb96ba69_0.zip', '/files/test.zip')

In [ ]:
yy

## The plot

In [ ]:
# set to min and max of data
vmin, vmax = min(data_mapped_filter['value']), max(data_mapped_filter['value'])

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(28,12))

# add a title and annotation
ax.set_title('FDI Inflow (Net, % of GDP)', fontdict={'fontsize': '25', 'fontweight' : '3'})

data_mapped_filter.plot(column='value',cmap='PuBu', linewidth=1, ax=ax, edgecolor='.5')
ax.axis('off')
sm = plt.cm.ScalarMappable(cmap='PuBu', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
cbar = fig.colorbar(sm)

In [ ]:
from matplotlib.animation import FuncAnimation
import time
import datetime as dt
vmin, vmax = np.quantile(data_mapped['value'], 0.05), np.quantile(data_mapped['value'], 0.95)
fig, ax = plt.subplots(1, figsize=(28,12))
ax.axis('off')
sm = plt.cm.ScalarMappable(cmap='PuBu', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
cbar = fig.colorbar(sm)

def animate(i):
    ax.clear()
    data_mapped_filter = data_mapped.loc[data_mapped.loc[:,'variable'] == f'YR{i}',:]
    ax.clear()
    ax.set_title(f'FDI Inflow (Net, % of GDP), year {i}', fontdict={'fontsize': '25', 'fontweight' : '3'})
    p = data_mapped_filter.plot(column='value',cmap='PuBu', linewidth=1, ax=ax, edgecolor='.5')
    return ax
    
anim = FuncAnimation(fig, animate, frames=range(2000,2019,), interval=500, blit=False)
current_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
anim.save(f'FDI_plot_{current_time}.gif')

## Storing the data

Storing the data in the Scientific Data Warehouse (SDW) can be done via the nuvolos package, which comes already installed in every JupyterLab application.

In this particular example, we will melt the data to be in the 'long' format, then we will store the resulting pandas DataFrame object in the SDW via a simple call. As a final step, we will provide some column comments and table comments to clarify the contents for later usage.

In [ ]:
FDI_GDP_PCT_data

In [ ]:
FDI_GDP_PCT_data_melt = FDI_GDP_PCT_data.melt(id_vars = ['economy', 'Country'])

In [ ]:
FDI_GDP_PCT_data_melt['variable'] = FDI_GDP_PCT_data_melt['variable'].apply(lambda x: x.replace('YR', '')).apply(lambda x: np.int_(x))

In [ ]:
import nuvolos
con = nuvolos.get_connection()
con.autocommit = True

In [ ]:
nuvolos.to_sql(df = FDI_GDP_PCT_data_melt, name = "FDI_GDP_PCT", con = con, if_exists='replace', index=False)

## Adding metadata

You can add table comments and column comments to provide a short description of the fields or tables of interest. While this feature cannot serve as a complete documentation solution, it will prove very useful for data used by a large number of users.

In [ ]:
cur = con.cursor()
cur.execute("COMMENT ON COLUMN FDI_GDP_PCT.value IS 'Foreign Direct Investment as Pct of GDP'")
cur.execute("COMMENT ON COLUMN FDI_GDP_PCT.variable IS 'Time Period'")
cur.execute("COMMENT ON COLUMN FDI_GDP_PCT.economy IS 'ISO A3 Country Code'")
cur.commit()